# Compare image preprocessing (difference of gaussians filter) vs. no preprocessing prior to running Tierpsy tracker

We're trying to use Tierpsy tracker to track motility of *C. elegans*.
Our early imaging experiments showed that bacterial lawns on plates cause dark streaks in the background of the image, making it difficult for computer vision software to reliably identify and track a worm.
In response, we imaged worms on empty plates (OP50-).
Even still, some fields of view (FOV) end up with chunks of agar, which cause dark backgrounds that make it difficult for the Tierpsy tracker to detect a worm.
This notebook explores whether applying a difference of Gaussians (DoG) filter before Tierpsy tracker analysis improves worm tracking.
The DoG filter suppresses the blurry, low-spatial-frequency background relative to the worms' sharp, high-frequency edges.

## Preprocessing microscopy imaging before Tierpsy tracker analysis

The preprocessing steps below produce Tierpsy tracker-ready MOVs with and without DoG filtering.

Convert the original nd2 into a TIFF stack. 
(I changed the parameters `XY_DOWNSAMPLE_FACTOR = 1` and `T_DOWNSAMPLE_FACTOR = 1`.)
I don't want to test for frame rate/size initially.

```
python scripts/convert_nd2.py convert-dir \
    --input-dirpath ~/s3/2024-worm-motility/microscopy_data/ \
    --output-dirpath processed-data/2024-worm-motility/microscopy_data/
```

DoG filter the TIFF stacks.
The script below applies a DoG filter to each TIFF in a stack individually and outputs the filtered stack.

```
python scripts/dog_filter2.py dog-filter-dir --input-dirpath processed-data --output-dirpath processed-data
```

Convert the DoG-filtered and non-DoG-filtered TIFFs to MOV for processing with Tierpsy tracker.
Tierpsy says it takes TIFF stacks, but I have been unsuccessful with this approach. 
It treats all TIFFs in a directory as a single file instead of each file as a stack.
MOV works fine.

```
python scripts/convert_tiff_to_mov2.py convert-dir \
    --input-dirpath processed-data \
    --output-dirpath processed-data \
    --filter-string Seq
```

Archive and compress the MOV files and transfer to a non-M1 mac to run Tierpsy tracker

```
find . -name "*.mov" | tar -cf microscopy_data.tar.gz -T -
```

## Tierpsy tracker analysis

The steps below produce worm motility measurements from DoG-filtered and non-DoG-filtered images.

Launch the Tierpsy tracker Docker container (double click on the `tierpsy.command` executable file) and run Tierpsy tracker.
I opened each file in the Tierpsy tracker GUI and optimized filter settings to retain the most worms from each FOV.
I then reconciled the optimal settings across the files and moved forward with these.
To reconcile values, I either:
* took an average of all optimal values for a given parameter or 
* took the minimum or maximum value for a given parameter

See the notebook `20240725-decide-tierpsy-batch-config-params.ipynb` for more information.

I then ran tierpsy tracker on all videos (dog filtered and non-dog filtered) with each of the parameters files.
```
tierpsy_process --video_dir_root local_drive/Downloads/NGM_manual_tracking_both/RawVideos --pattern_include *mov --json_file local_drive/Downloads/NGM_manual_tracking_both/nofilter.json

tierpsy_process --video_dir_root local_drive/Downloads/NGM_manual_tracking_both/RawVideos --pattern_include *mov --json_file local_drive/Downloads/NGM_manual_tracking_both/dogfilter.json
```

## look at images for ground truth

### 1 hr images

- 20240722_104547_685: 1 worm inframe on agar the entire time; 1 worm in and out of FOV
- 20240722_104800_780: 1 worm inframe on agar the entire time
- 20240722_105049_233: 1 worm inframe no agar the entire time, blurry; 1 worm enters FOV, blurry
- 20240722_105204_511: 1 worm in frame but FOV moves, no agar until end

### 5 hr images
- 20240722_135936_253: 1 worm inframe on agar entire time
- 20240722_140218_590: 1 worm in frame no agar the entire time
- 20240722_140333_297: 1 worm in frame on agar the entire time
- 20240722_140446_147: 2 worms in frame on agar the entire time
- 20240722_140634_714: 2 worms on agar the entire time. Just their heads exit the frame. 1 worm no agar the entire time.


**Total possible**: 14 worms.  
**In FOV entire time**: 12 worms

In [53]:
key <- data.frame(file_id = c("20240722_104547_685", "20240722_104800_780", "20240722_105049_233", "20240722_105204_511",
                             "20240722_135936_253", "20240722_140218_590", "20240722_140333_297", "20240722_140446_147",
                              "20240722_140634_714"),
                  time = c("1hr", "1hr", "1hr", "1hr", "5hr", "5hr", "5hr", "5hr", "5hr"),
                  description = c("1 worm inframe on agar the entire time; 1 worm in and out of FOV", 
                                  "1 worm inframe on agar the entire time",
                                  "1 worm inframe no agar the entire time, blurry; 1 worm enters FOV, blurry",
                                  "1 worm in frame but FOV moves, no agar until end",
                                  "1 worm inframe on agar entire time",
                                  "1 worm in frame no agar the entire time",
                                  "1 worm in frame on agar the entire time",
                                  "2 worms in frame on agar the entire time",
                                  "2 worms on agar the entire time. Just their heads exit the frame; 1 worm no agar the entire time."))

key

file_id,time,description
<chr>,<chr>,<chr>
20240722_104547_685,1hr,1 worm inframe on agar the entire time; 1 worm in and out of FOV
20240722_104800_780,1hr,1 worm inframe on agar the entire time
20240722_105049_233,1hr,"1 worm inframe no agar the entire time, blurry; 1 worm enters FOV, blurry"
20240722_105204_511,1hr,"1 worm in frame but FOV moves, no agar until end"
20240722_135936_253,5hr,1 worm inframe on agar entire time
20240722_140218_590,5hr,1 worm in frame no agar the entire time
20240722_140333_297,5hr,1 worm in frame on agar the entire time
20240722_140446_147,5hr,2 worms in frame on agar the entire time
20240722_140634_714,5hr,2 worms on agar the entire time. Just their heads exit the frame; 1 worm no agar the entire time.


## Set up notebook

In [1]:
library(dplyr)
library(purrr)
library(rhdf5)
library(stringr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
setwd("..")

## Functions

In [44]:
process_hdf5_files <- function(hdf5_files, required_frames) {
  hdf5_files %>%
    map_dfr(~{
      # The "/timeseries_data" contains per-worm and per-frame measurements (ex curvature_mean_tail_abs_IQR).
      # Susceptible to NAs, but NAs might be useful to winnow to high-signal data.
      # This is probably the main readout we'll use for estimating phenotypes.
      timeseries_data <- h5read(.x, name = "/timeseries_data")
      
      # We need a distinct worm_index for each worm.
      # Grab folder that nd2 sits in, which right now is our unique identifier.
      # This might need to change for subsequent experiments.
      # file_id <- str_extract(basename(dirname(.x)), "^[^\\.]+")
      # just parse file_id after bc it will change per experiment
        
      processed_data <- timeseries_data %>%
        mutate(file_id = .x,
               across(everything(), ~replace(., is.nan(.), NA))) %>%
        select(-well_name) %>%  # Remove well_name column
        filter(!if_all(-c(worm_index, timestamp), is.na))  # Filter out rows that are all NA except for the first two columns
      
      # Filter based on number of frames.
      # If there aren't enough frames of the worm, it usually means that Tierpsy tracker had trouble measuring that worm.
      # When this is the case, the data have a lot of NAs.
      # Alternatively, few frames could mean that the worm entered into the field of view during mid-capture.
      # I'm not sure how to compare mid-capture worms against normal worms, so for now we don't discriminate between these two failure modes,
      # we just remove everything that doesn't have plentiful frames.
      processed_data %>%
        group_by(worm_index) %>%
        filter(n() > required_frames) %>%
        ungroup() 
    })
}

In [45]:
parse_file_id <- function(df) {
 df %>%
    mutate(file_id = str_extract(basename(dirname(file_id)), "^[^\\.]+"))
}

## Read in time series data

In [43]:
hdf5_files_dogfilter_imgs_with_dogfilter_json <- Sys.glob("tmp/NGM_manual_tracking_both/dogfilter_json_results/Results/*/*/*dogfilter_featuresN.hdf5")
hdf5_files_nofilter_imgs_with_dogfilter_json <- Sys.glob("tmp/NGM_manual_tracking_both/dogfilter_json_results/Results/*/*/*0_featuresN.hdf5")

hdf5_files_dogfilter_imgs_with_nofilter_json <- Sys.glob("tmp/NGM_manual_tracking_both/nofilter_json_results/Results/*/*/*dogfilter_featuresN.hdf5")
hdf5_files_nofilter_imgs_with_nofilter_json <- Sys.glob("tmp/NGM_manual_tracking_both/nofilter_json_results/Results/*/*/*0_featuresN.hdf5")

In [46]:
# using required_frames = 200 means that a worm must show up in 1/3 of the frames for it to be tracked and counted
dogfilter_img_dogfilter_json <- process_hdf5_files(hdf5_files_dogfilter_imgs_with_dogfilter_json, required_frames = 200) %>% parse_file_id()
nofilter_img_dogfilter_json <- process_hdf5_files(hdf5_files_nofilter_imgs_with_dogfilter_json, required_frames = 200) %>% parse_file_id()
dogfilter_img_nofilter_json <- process_hdf5_files(hdf5_files_dogfilter_imgs_with_nofilter_json, required_frames = 200) %>% parse_file_id()
nofilter_img_nofilter_json <- process_hdf5_files(hdf5_files_nofilter_imgs_with_nofilter_json, required_frames = 200) %>% parse_file_id

In [56]:
dogfilter_img_dogfilter_json %>%
  group_by(file_id, worm_index) %>%
  tally() %>%
  full_join(key) %>%
  arrange(time)

Joining with `by = join_by(file_id)`


file_id,worm_index,n,time,description
<chr>,<int[1d]>,<int>,<chr>,<chr>
20240722_104547_685,2,369,1hr,1 worm inframe on agar the entire time; 1 worm in and out of FOV
20240722_104800_780,1,735,1hr,1 worm inframe on agar the entire time
20240722_105049_233,NA,NA,1hr,"1 worm inframe no agar the entire time, blurry; 1 worm enters FOV, blurry"
20240722_105204_511,NA,NA,1hr,"1 worm in frame but FOV moves, no agar until end"
20240722_135936_253,8,546,5hr,1 worm inframe on agar entire time
20240722_140333_297,1,735,5hr,1 worm in frame on agar the entire time
20240722_140446_147,3,735,5hr,2 worms in frame on agar the entire time
20240722_140218_590,NA,NA,5hr,1 worm in frame no agar the entire time
20240722_140634_714,NA,NA,5hr,2 worms on agar the entire time. Just their heads exit the frame; 1 worm no agar the entire time.


**Score**: 1/2, 1/1, 0/2, 0/1; 1/1, 1/1, 1/1, 1/2, 0/1, 0/3 = **6/14 worms**

In [58]:
nofilter_img_nofilter_json %>%
  group_by(file_id, worm_index) %>%
  tally() %>%
  full_join(key) %>%
  arrange(time)

Joining with `by = join_by(file_id)`


file_id,worm_index,n,time,description
<chr>,<int[1d]>,<int>,<chr>,<chr>
20240722_105204_511,1,735,1hr,"1 worm in frame but FOV moves, no agar until end"
20240722_104547_685,NA,NA,1hr,1 worm inframe on agar the entire time; 1 worm in and out of FOV
20240722_104800_780,NA,NA,1hr,1 worm inframe on agar the entire time
20240722_105049_233,NA,NA,1hr,"1 worm inframe no agar the entire time, blurry; 1 worm enters FOV, blurry"
20240722_140218_590,1,339,5hr,1 worm in frame no agar the entire time
20240722_140218_590,2,396,5hr,1 worm in frame no agar the entire time
20240722_140333_297,3,735,5hr,1 worm in frame on agar the entire time
20240722_140634_714,1,736,5hr,2 worms on agar the entire time. Just their heads exit the frame; 1 worm no agar the entire time.
20240722_140634_714,2,736,5hr,2 worms on agar the entire time. Just their heads exit the frame; 1 worm no agar the entire time.


**Score**: 1/1, 0/2, 0/1, 0/2; (one worm split in two), 1/1, 2/3, 0/1, 0/2 = **4/14**

In [57]:
nofilter_img_dogfilter_json %>%
  group_by(file_id, worm_index) %>%
  tally() %>%
  full_join(key) %>%
  arrange(time)

Joining with `by = join_by(file_id)`


file_id,worm_index,n,time,description
<chr>,<int[1d]>,<int>,<chr>,<chr>
20240722_104800_780,8,421,1hr,1 worm inframe on agar the entire time
20240722_105204_511,1,735,1hr,"1 worm in frame but FOV moves, no agar until end"
20240722_104547_685,NA,NA,1hr,1 worm inframe on agar the entire time; 1 worm in and out of FOV
20240722_105049_233,NA,NA,1hr,"1 worm inframe no agar the entire time, blurry; 1 worm enters FOV, blurry"
20240722_140218_590,1,339,5hr,1 worm in frame no agar the entire time
20240722_140218_590,2,396,5hr,1 worm in frame no agar the entire time
20240722_140333_297,3,735,5hr,1 worm in frame on agar the entire time
20240722_140333_297,6,527,5hr,1 worm in frame on agar the entire time
20240722_140333_297,696,207,5hr,1 worm in frame on agar the entire time


**Score**: 1/1, 1/1, 0/2, 0/2; (one worm split into 2), (one worm split into 3), 1/2, 2/3, 0/1 = **5/14** (i'm not counting the worms that get split)

In [59]:
dogfilter_img_nofilter_json %>%
  group_by(file_id, worm_index) %>%
  tally() %>%
  full_join(key) %>%
  arrange(time)

Joining with `by = join_by(file_id)`


file_id,worm_index,n,time,description
<chr>,<int[1d]>,<int>,<chr>,<chr>
20240722_104547_685,2,278,1hr,1 worm inframe on agar the entire time; 1 worm in and out of FOV
20240722_104800_780,1,312,1hr,1 worm inframe on agar the entire time
20240722_105049_233,NA,NA,1hr,"1 worm inframe no agar the entire time, blurry; 1 worm enters FOV, blurry"
20240722_105204_511,NA,NA,1hr,"1 worm in frame but FOV moves, no agar until end"
20240722_135936_253,6,388,5hr,1 worm inframe on agar entire time
20240722_140446_147,2,735,5hr,2 worms in frame on agar the entire time
20240722_140218_590,NA,NA,5hr,1 worm in frame no agar the entire time
20240722_140333_297,NA,NA,5hr,1 worm in frame on agar the entire time
20240722_140634_714,NA,NA,5hr,2 worms on agar the entire time. Just their heads exit the frame; 1 worm no agar the entire time.


**Score**: 1/2, 1/1, 0/2, 0/1; 1/1, 1/2, 0/1, 0/1, 0/3 = **4/14 worms**

# Conclusion

**DoG-filtered images with a DoG-specific Tierpsy tracker config file lead to the best worm recall.**
Currently, think the only strategy that would out-compete this is making a per-image (image-specific) config file and running Tierpsy tracker with that.

Alternatively, we could perform more sophisticated masking/filtering/segmentation before running Tierpsy tracker.

In [60]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/wormmotility/lib/libopenblasp-r0.3.27.dylib;  LAPACK version 3.12.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] stringr_1.5.1 rhdf5_2.46.1  purrr_1.0.2   dplyr_1.1.4  

loaded via a namespace (and not attached):
 [1] crayon_1.5.3        vctrs_0.6.5         cli_3.6.3          
 [4] rlang_1.1.4         stringi_1.8.4       generics_0.1.3     
 [7] jsonlite_1.8.8      glue_1.7.0          htmltools_0.5.8.1  
[10] IRdisplay_1.1       IRkernel_1.3.2      fansi_1.0.6        
[13] evaluate_0.24.0     tibble_3.2.1        fastmap_1.2.0      
[16] base64enc_0.1-3     Rhdf5lib_1.24.0     lifecycle_1.0.4  